In [1]:
import pandas as pd
dfs = pd.read_excel('tables\\Basin MT Tables.xlsx', sheet_name=None)

# GSA = "James GSA"
GSA = "SKGSA"

public_wells = dfs['Public Wells'].query('GSA == @GSA')
ps_codes = [i for i in public_wells['PS Code'].unique()]

# chemical_names = dfs['chemical names'].query('GSA == @GSA')
chemical_names = dfs['chemical crosswalk'].query('GSA == @GSA')
# chemical_names = chemical_names['Chemical Name'].unique()
chemicals_to_check = [i for i in chemical_names['public_name']]
# chemicals_to_check

In [2]:
# df
# ps_codes

Conversions

In [4]:
import polars as pl
possible_units = pl.read_parquet(
	r"C:\Users\Denver\Downloads\SWRCB_WQ\swrcb_wq.parquet"
	).select(pl.col("Units of Measure").str.strip())
possible_units = possible_units['Units of Measure'].unique().to_list()
possible_units

['AGGR',
 '',
 'PCI/L',
 'MFL',
 'TON',
 'UMHO/CM',
 'NG/L',
 'LANG',
 'MG/L',
 'UG/L',
 'PH',
 'NTU',
 'C',
 'UNITS']

MCL creation

In [5]:
import polars as pl

df = pl.read_parquet(
	r"C:\Users\Denver\Downloads\SWRCB_WQ\swrcb_wq.parquet"
	).filter(
		# pl.col('PS Code').str.strip() == f'{id}'
		pl.col('PS Code').str.strip().is_in(ps_codes)
		).select(
			pl.col(pl.Utf8).str.strip()
			).with_columns(
				pl.col("Sample Date").str.strptime(pl.Date, fmt="%m-%d-%Y").alias("Date")
				).filter(
					pl.col('Date') > pl.date(2015,1,1)
					).filter(
						pl.col('Date') < pl.date(2022,1,1)
						)
					


In [10]:
import pint
u = pint.UnitRegistry()

conversion_names = {
	'MG/L':'milligram / liter',
	'NG/L':'nanogram / liter',
	'UG/L':'microgram / liter',
}


In [12]:
# print(df['Analyte Name'].unique().to_list())
# chems = [
# 	'ARSENIC',
# 	"NITRATE",
# 	'1,2-DIBROMO-3-CHLOROPROPANE',
# 	'1,2,3-TRICHLOROPROPANE',
# 	]
chems = chemicals_to_check
well = df.filter(
	pl.col('Analyte Name').is_in(chems)
# ).with_columns(pl.col('Result').cast(pl.Float32,strict=True).fill_null("ND").alias("reading"))
# make ND = -1 so that it can be a float
).with_columns(pl.col('Result').cast(pl.Float32,strict=True).fill_null(-1).alias("reading"))

well.with_columns(pl.col("Units of Measure").map_dict(conversion_names, default=pl.col("Units of Measure")).alias("Units"))

# .alias("Units")
# chem_readings = well.to_pandas()
# chem_readings['Units of measure']
# replace values in Units of Measure with the conversion_names
# well = well.with_columns(pl.col('Units of Measure').str.replace('MG/L', 'milligram / liter').alias('Units of Measure'))
# well.select(pl.col("Units of Measure"))

Regulating Agency,Water System Number,Water System Name,System Status,Water System Classification,Principal County Served,Population Served,Service Connections,PS Code,Sampling Point Name,Facility Type,Facility Status,Sample Type,Sample Date,Sample Time,Analysis Date,ELAP Cert#,Lab Name,Analyte Code,Analyte Name,Result,Counting Error,Units of Measure,Less Than Reporting Level,Reporting Level,DLR,MCL,Method,Date,reading,Units
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,date,f32,str
"""DISTRICT 23 - ...","""CA1010006""","""CITY OF FOWLER...","""A""","""C""","""FRESNO""","""6700""","""2016""","""CA1010006_007_...","""WELL 06""","""WL""","""A""","""RT""","""04-13-2016""","""15:45:00""","""04-19-2016""","""1180""","""BSK ANALYTICAL...","""2984""","""TRICHLOROETHYL...",null,null,"""UG/L""","""Y""","""0.500000000""",""".5""","""5""",null,2016-04-13,-1.0,"""microgram / li..."
"""DISTRICT 23 - ...","""CA1010006""","""CITY OF FOWLER...","""A""","""C""","""FRESNO""","""6700""","""2016""","""CA1010006_007_...","""WELL 06""","""WL""","""A""","""RT""","""06-22-2018""","""10:00:00""","""06-26-2018""","""1180""","""BSK ANALYTICAL...","""2414""","""1,2,3-TRICHLOR...",null,null,"""UG/L""","""Y""","""0.000000000""",null,"""0.005""",null,2018-06-22,-1.0,"""microgram / li..."
"""DISTRICT 23 - ...","""CA1010006""","""CITY OF FOWLER...","""A""","""C""","""FRESNO""","""6700""","""2016""","""CA1010006_007_...","""WELL 06""","""WL""","""A""","""RT""","""07-18-2016""","""13:49:00""","""07-25-2016""","""1180""","""BSK ANALYTICAL...","""2931""","""1,2-DIBROMO-3-...","""0.0290000""",null,"""UG/L""","""N""","""0.000000000""",""".01""",""".2""",null,2016-07-18,0.029,"""microgram / li..."
"""DISTRICT 23 - ...","""CA1010006""","""CITY OF FOWLER...","""A""","""C""","""FRESNO""","""6700""","""2016""","""CA1010006_007_...","""WELL 06""","""WL""","""A""","""RT""","""07-27-2015""","""13:55:00""","""07-31-2015""","""1180""","""BSK ANALYTICAL...","""2931""","""1,2-DIBROMO-3-...","""0.0140000""",null,"""UG/L""","""N""","""0.000000000""",""".01""",""".2""",null,2015-07-27,0.014,"""microgram / li..."
"""DISTRICT 23 - ...","""CA1010006""","""CITY OF FOWLER...","""A""","""C""","""FRESNO""","""6700""","""2016""","""CA1010006_007_...","""WELL 06""","""WL""","""A""","""RT""","""08-01-2017""","""10:45:00""","""08-03-2017""","""1180""","""BSK ANALYTICAL...","""2931""","""1,2-DIBROMO-3-...",null,null,"""UG/L""","""Y""","""0.000000000""",""".01""",""".2""",null,2017-08-01,-1.0,"""microgram / li..."
"""DISTRICT 23 - ...","""CA1010006""","""CITY OF FOWLER...","""A""","""C""","""FRESNO""","""6700""","""2016""","""CA1010006_007_...","""WELL 06""","""WL""","""A""","""RT""","""09-21-2018""","""08:30:00""","""09-23-2018""","""1180""","""BSK ANALYTICAL...","""2414""","""1,2,3-TRICHLOR...",null,null,"""UG/L""","""Y""","""0.000000000""",null,"""0.005""",null,2018-09-21,-1.0,"""microgram / li..."
"""DISTRICT 23 - ...","""CA1010006""","""CITY OF FOWLER...","""A""","""C""","""FRESNO""","""6700""","""2016""","""CA1010006_008_...","""WELL 07""","""WL""","""A""","""RT""","""01-04-2018""","""14:40:00""","""01-05-2018""","""1180""","""BSK ANALYTICAL...","""1040""","""NITRATE""","""3.1000000""",null,"""MG/L""","""N""","""0.400000000""",""".4""","""10""",null,2018-01-04,3.1,"""milligram / li..."
"""DISTRICT 23 - ...","""CA1010006""","""CITY OF FOWLER...","""A""","""C""","""FRESNO""","""6700""","""2016""","""CA1010006_008_...","""WELL 07""","""WL""","""A""","""RT""","""01-26-2015""","""08:30:00""","""01-29-2015""","""1180""","""BSK ANALYTICAL...","""2931""","""1,2-DIBROMO-3-...","""0.2000000""",null,"""UG/L""","""N""","""0.000000000""",""".01""",""".2""",null,2015-01-26,0.2,"""microgram / li..."
"""DISTRICT 23 - ...","""CA1010006""","""CITY OF FOWLER...","""A""","""C""","""FRESNO""","""6700""","""2016""","""CA1010006_008_...","""WELL 07""","""WL""","""A""","""RT""","""04-13-2016""","""13:45:00""","""04-14-2016""","""1180""","""BSK A

In [ ]:


# well[['PS Code','Date','Analyte Name','Result','Units of Measure']].sort('Date',reverse=True).to_pandas()


mcl_values = well.pivot(
	values='reading',
	index='PS Code',
	columns='Analyte Name',
	aggregate_fn='max'
).to_pandas().replace(-1,"ND").fillna("NA")

# .to_clipboard()
# .select(pl.col(pl.Float32).str.replace("-1","ND"))
# .fill_null("NA")

#.sort('Date',reverse=True)#.to_pandas().to_clipboard()

Results

In [ ]:
import polars as pl

# id = 'CA1000018_001_001'
id = 'CA1010029_022_022'
# Analyte Name
# pl.read_parquet(r"C:\Users\Denver\Downloads\SWRCB_WQ\swrcb_wq.parquet").filter(pl.col('Water System Number').str.strip() == f'{id}').select(pl.col(pl.Utf8).str.strip()).with_columns(pl.col("Sample Date").str.strptime(pl.Date, fmt="%m-%d-%Y").alias("Date")).filter(pl.col('Date') > pl.date(2015,1,1))
df = pl.read_parquet(
	r"C:\Users\Denver\Downloads\SWRCB_WQ\swrcb_wq.parquet"
	).filter(
		# pl.col('PS Code').str.strip() == f'{id}'
		pl.col('PS Code').str.strip().is_in(ps_codes)
		).select(
			pl.col(pl.Utf8).str.strip()
			).with_columns(
				pl.col("Sample Date").str.strptime(pl.Date, fmt="%m-%d-%Y").alias("Date")
				).filter(
					pl.col('Date') > pl.date(2015,1,1)
					)


In [ ]:
chems = chemicals_to_check
well = df.filter(
	pl.col('Analyte Name').is_in(chems)
).with_columns(pl.col('Result').cast(pl.Float32,strict=True).fill_null("ND").alias("reading"))
# well[['PS Code','Date','Analyte Name','Result','Units of Measure']].sort('Date',reverse=True).to_pandas()
well.pivot(
	values='reading',
	index='Date',
	columns='Analyte Name',
	aggregate_fn='max'
).sort('Date',reverse=True)#.to_pandas().to_clipboard()